### Coweeta, NC RHESSys workflow

US Forest Service WS18 is a 12.3ha, mixed hardwood-oak, control catchment that’s been undisturbed since 1927.


RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).

This notebook is built on RHESSys sample workflow. Not all steps are documented. Here we focus on explaining new or modifications.

Users interested in seeing step outputs, remove **output = ** from the command line.


In [ ]:
import os
import logging
import hs_utils
from rhessys_wf import *
%matplotlib inline

In [ ]:
w = RHESSysWorkflow(project_name='coweeta_ncV3', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

### Retrieve Coweeta, NC data from HydroShare Resource

### Step 1 of 3

The first step to retrieve a resource from HydroShare is to create a HydroShare connection as follows:

In [ ]:
hs = hs_utils.hydroshare()

### Step 2 of 3

Dr. Laurence Lin from UNC Institute for the Environment
has prepared a HydroShare Resource for **Coweeta, NC** [(RHESSys (5.20) watershed 18, Coweeta, NC)](https://www.hydroshare.org/resource/c397f68fbc9847d6a717b0a2c2fe7671/) 

If you follow the web link, or move your mouse cursor over the above web link, you will note the web link has nas numbers and text after **https://www.hydroshare.org/resource/**. This text string is a resource id. Here the resource id is **c397f68fbc9847d6a717b0a2c2fe7671** and is needed to retrieve the resource prepared by Dr. Laurence Lin.

In [ ]:
resource_id = 'c397f68fbc9847d6a717b0a2c2fe7671'
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)

### Step 3 of 3

After the resource has finished downloading, the next step is to unzip the folder in your workspace.
We know the name of the zipped file "ws18.zip" from the above step. If you are using a different resource, you will be required to change **hs.content['ws18.zip']**, as well as **ws18** folder names below.


In [ ]:
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = hs.content['ws18.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print 'Steps Finished'

This step **registers the DEM.tif** stored in the resource rather than using the `get_USGSDEMForBoundingbox` command.

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"
demfile = w.sub_project_folder + '/RHESSys_ETV/ws18/DEM.tif'
options = " -p " + w.sub_project_folder + " -d " + demfile + " -b \"CWT18 DEM\""
w.RegisterDEM(w.sub_project_folder, demfile)

**Register streamflow gage** coordinates from a point shapefile into metadata 
store for a project directory, copying the shapefile into the project directory 
in the process

In [ ]:
gagefile = w.sub_project_folder + '/RHESSys_ETV/ws18/gage.shp'
layername = 'gage'
id_attribute = 'gage_id'
id_value = "\"Weir 18\""
w.RegisterGage(w.sub_project_folder, gagefile, layername, id_attribute, id_value)


In [ ]:
output = w.get_USGSNLCDForDEMExtent(w.sub_project_folder)

**Register raster** into metadata store for a project directory,
copying the raster into the project directory in the process.  Raster type must
be one of GenericMetadata.RASTER_TYPES.

In [ ]:
roads_file = w.sub_project_folder + '/RHESSys_ETV/ws18/rhessys/roads.tif'
options = " -p " + w.sub_project_folder + " -t roads " + " -r " + roads_file + " --force -s near "
w.RegisterRasterOptions(options)

In [ ]:
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)

Here the resource or user supplied **LAI** is registered.

In [ ]:
lai_file = w.sub_project_folder + '/RHESSys_ETV/ws18/lai.tif'
options = " -p " + w.sub_project_folder + " -t lai " + " -r " + lai_file + " -s bilinear --force "
w.RegisterRasterOptions(options)

Here the resource or user supplied **ISOHYET** is registered.

In [ ]:
isohyet_file = w.sub_project_folder + '/RHESSys_ETV/ws18/isohyet.tif'
options = " -p " + w.sub_project_folder + " -t isohyet " + " -r " + isohyet_file + " --force -s bilinear "
w.RegisterRasterOptions(options)

In [ ]:
output = w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)

In [ ]:
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"RHESSys model for watershed 18 at Coweeta LTER"')

The `ImportRHESSysSource` function downloads and compiles the latest [RHESSys](https://github.com/RHESSys/RHESSys) source code in the user defined project folder. This make take a couple minutes.   

In [ ]:
output = w.ImportRHESSysSource(w.sub_project_folder)

In [ ]:
cell_size = 250
area_estimate = 0.13
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'lai', 'bilinear')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'isohyet', 'bilinear')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'roads', 'nearest')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'landcover', 'nearest')

In [ ]:
output = w.RegisterLandcoverReclassRules(w.sub_project_folder)

In [ ]:
options = " -p " + w.sub_project_folder + " --skipRoads"
output = w.GenerateLandcoverMapsOptions(options)

In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder, options='')

In [ ]:
patch_options = " -p " + w.sub_project_folder + " -t grid"
output = w.GeneratePatchMapOptions(patch_options)

In [ ]:
w.climate_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/ws18/rhessys/clim'
output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)

In [ ]:
options = " -p " + w.sub_project_folder + " -c cwt"
output = w.GenerateWorldTemplateOptions(options)


In [ ]:
output = w.CreateWorldfile(w.sub_project_folder)

**Flowtable** will be created in the RHESSys folder of the project directory.

In [ ]:
options = " -p " + w.sub_project_folder + " --routeRoads"
output = w.CreateFlowtableOptions(options)

In [ ]:
output = w.RunLAIRead(w.sub_project_folder)

In [ ]:
output = w.RunCmd(w.sub_project_folder, 3)

The RunModel function run the RHESSys simulation! Please wait a couple minutes.

In [ ]:
output = w.RunModel(w.sub_project_folder)

## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)
  